## Decision Tree

In [1]:
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split

X, y = datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [2]:
y
np.bincount(y)

array([212, 357])

In [3]:
# Prepare Node class to presist node info

class Node:
    def __init__(self,spliting_feature_idx=None, spliting_label=None, left=None,right=None,*, value=None):
        self.spliting_feature_idx= spliting_feature_idx
        self.spliting_label= spliting_label
        self.left= left
        self.right= right
        self.value= value

    def is_leaf_node(self):
        return self.value is not None
    


In [4]:
from collections import Counter

max_depth = 50
min_samples_split = 2

def fit(X,y):
    root_node = branches(X,y)
    return root_node


def branches(X,y, depth=0):

    n_samples = X.shape[0]
    n_labels = len(np.unique(y))

    # check the stopping criteria
    if (depth>= max_depth or n_labels==1 or n_samples<min_samples_split):
        leaf_value = most_common_label(y)
        return Node(value=leaf_value)
    
    
    feature_idx,label,left_idx,right_idx = find_best_split(X,y)

    # recursively go deep to reach till end of left & right leaf node
    left_node = branches(X[left_idx, :], y[left_idx], depth+1) if len(left_idx) >0 else None
    right_node = branches(X[right_idx, :], y[right_idx], depth+1) if len(right_idx) >0 else None

    # prepare node and reslove recursive stock backword
    return Node(feature_idx, label, left_node, right_node)


def most_common_label(y):
    counter = Counter(y)
    value = counter.most_common(1)[0][0]
    return value


def find_best_split(X,y):
    parent_E = entropy(y)
    features = X.shape[1]

    best_split_info = None
    best_IG = -1
    
    for f_idx in range(features):
        feature = X[:, f_idx] # slice particular feature
        labels = np.unique(feature)

        for label in labels:
            left_idx, right_idx = split_column(feature,label)

            l_E = entropy(y[left_idx])
            r_E = entropy(y[right_idx])
            l_W = len(left_idx) / len(y)
            r_W = len(right_idx) / len(y)
            
            child_E = (l_W * l_E)+(r_W * r_E)
            IG = parent_E - child_E

            if(best_IG < IG):
                best_IG = IG
                best_split_info = (f_idx, label, left_idx,right_idx)

    
    return best_split_info

def entropy(y):
    # y contain only 0 and 1. result - [no_zeros, no_ones]
    labels_count = np.bincount(y) # skip zero since zero always px multiply
    px = labels_count / len(y) 
    return -np.sum([p * np.log(p) for p in px if p>0])

def split_column(cols, value):
    left_idx = np.argwhere(cols<=value).flatten()
    right_idx = np.argwhere(cols>value).flatten()
    return left_idx, right_idx



root_node = fit(X_train,y_train)

In [13]:
def predict(X, tree):
    return np.array([traverse_tree(x, tree) for x in X])

def traverse_tree(x, node):
    if node.is_leaf_node():
        return node.value
    
    if x[node.spliting_feature_idx] <= node.spliting_label:
        return traverse_tree(x, node.left)
    return traverse_tree(x, node.right)
    

In [19]:
predictions = predict(X_test,root_node)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, predictions)
print(acc)

0.9298245614035088


## Random Forest 

#### collection Decision Tree created forest here :P 

In [16]:
# save previous method to dictionary 
decision_tree_fit = fit
decision_tree_predict = predict

n_tree = 20
def create_forest(X,y):
    forest = []
    for _ in range(n_tree):
         # create random sample data for each tree
         n_data = X.shape[0]
         idx = np.random.choice(n_data, int(n_data* .7), replace=True)
         t = decision_tree_fit(X[idx], y[idx])
         forest.append(t)
    return forest

forest = create_forest(X,y)
    

In [26]:
forest_pred = np.swapaxes([decision_tree_predict(X_test, tree) for tree in forest], 0, 1)

 

predictions = np.array([most_common_label(p) for p in forest_pred ])

predictions.shape, y_test.shape

acc = accuracy(y_test, predictions)
print(acc)

1.0
